In [94]:
# Assumptions 
# - 2x farm where one asset is stablecoin, e.g.  ETH/USDC
# - there is always borrow availability
# - borrow rate, trading fees, rewards are all constant


from math import exp, sqrt
from statistics import mean
import pandas as pd 


class Agent():
    
    def __init__(self, equity, price, borrow_rate, farming_rate):
        print("init")
        
        self.last_price = price
        self.borrow_rate = borrow_rate
        self.farming_rate = farming_rate
        
        # Make an intial investment - 50% stablecoin, 50% borrowed shitcoin
        self.equity = equity
        self.assetA_count = equity / price  # shitcoin
        self.assetB_count = equity          # stablecoin
        self.debt_value =  self.assetA_count * price

        # Settings for naive rebalancer - trigger when price moves 10% in either direction
        self.upper = price * 1.1
        self.lower = price * 0.9
        
    def rebalance(self):
        print("rebalance")
        
        # update self.debt_value
        
    def calculate_position(self, new_price):
        print("calculate_pnl")
        
        price_change_ratio = new_price / self.last_price
        lp_multiplier = sqrt(price_change_ratio)
        
        # Taken from https://docs.google.com/spreadsheets/d/1NYVm1aj8cpMOvlsv4Y_JmTrO7X59QnJnH2jDSfZM2zg
        self.position_value = (2*self.equity*lp_multiplier) + 2*mean([self.equity*lp_multiplier, self.equity])*(exp(self.farming_rate)-1)
        
        self.debt_value = self.assetA_count * new_price * exp(self.borrow_rate)
        self.equity = self.position_value - self.debt_value
        
        print ("price_change_ratio=" + str(price_change_ratio))
        print ("lp_multiplier=" + str(lp_multiplier))
        print ("position_value=" + str(self.position_value))
        print ("debt_value=" + str(self.debt_value))
        print ("equity_value=" + str(self.equity))

    def action(self, new_price):
        print ("\n")
        print ("ACTION...  price is " + str(new_price))
        
        self.calculate_position(new_price)
        self.last_price = new_price
        
     #    Rebalance if necessary
     #   if ((price > self.upper) or (price < self.lower)):
     #       self.rebalance()
     #       self.calculate_position()
            
    
        

# RUN SIMULATION
df = pd.DataFrame(columns = ['step', 'price', 'position_value', 'debt_value', 'equity_value', 'rebalanced'])
price = [13.70, 13.70, 14.385, 15.07, 15.76, 11.65, 12.33]

#price = [13.70, 14.39]

INITIAL_EQUITY = 10000
BORROW_APR = 85        # token borrow rate (%)
FARMING_APR = 100      # farming and all rewards rate (%)


# Assume a time step is 1 day
agent = Agent(
    INITIAL_EQUITY,        # initial equity
    price.pop(0),          # initial token_price 
    BORROW_APR/365/100,    # borrow rate per time step 
    FARMING_APR/365/100    # farming rate at 1x leverage per time step
)

step = 0
for p in price:
    agent.action(p) 
    
    # Record position
    
    df = df.append ({
        'step': step,
        'price': p,
   #     'position_value': agent.position_value,
   #     'debt_value': agent.debt_value,
   #     'equity_value': agent.equity_value,
   #     'rebalanced': agent.rebalanced
    },
        ignore_index=True)
    
    step += 1


df.head()


# UNUSED
# entry_fee = 0
# exit_fee = 0
# controller_fee = 0.001  
# reinvestent_fee = 0.039 
# tx_fee = 0  # network fee
# performance_fee = 0



#TODO
# repeated iterations of action - calcs are working for 1 step but need to update state - OR only update state on rebalance when you crystalise loss, at other times keep track of timesteps since last rebalance to calculate equity etc.
# put in real price data
# do rebalances
            


init


ACTION...  price is 13.7
calculate_pnl
price_change_ratio=1.0
lp_multiplier=1.0
position_value=20054.869650130822
debt_value=10023.314808075489
equity_value=10031.554842055333


ACTION...  price is 14.385
calculate_pnl
price_change_ratio=1.05
lp_multiplier=1.02469507659596
position_value=20614.292147921147
debt_value=10524.480548479265
equity_value=10089.811599441882


ACTION...  price is 15.07
calculate_pnl
price_change_ratio=1.0476190476190477
lp_multiplier=1.023532631438318
position_value=20710.51668940804
debt_value=11025.646288883041
equity_value=9684.870400524998


ACTION...  price is 15.76
calculate_pnl
price_change_ratio=1.0457863304578632
lp_multiplier=1.0226369494878733
position_value=19861.954660117437
debt_value=11530.470173377353
equity_value=8331.484486740084


ACTION...  price is 11.65
calculate_pnl
price_change_ratio=0.7392131979695432
lp_multiplier=0.8597750856878461
position_value=14368.914980471762
debt_value=8523.475730954706
equity_value=5845.439249517056




,step,price,position_value,debt_value,equity_value,rebalanced
0,0.0,13.700,NaN,NaN,NaN,NaN
1,1.0,14.385,NaN,NaN,NaN,NaN
2,2.0,15.070,NaN,NaN,NaN,NaN
3,3.0,15.760,NaN,NaN,NaN,NaN
4,4.0,11.650,NaN,NaN,NaN,NaN
